In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pathlib as plib
import sys
import time
import utils
import cp_optimizer
import math

from ortools.sat.python import cp_model

In [2]:
q_list = ["q1", "q3", "q2", "q4","q2", "q5", "q6"]
df_queries = utils.load(q_list, num_partitions='16')
res = utils.interpolate(df_queries)
precision = 10
C =16
C_ = None # for split
Q = len(q_list)
R = int(round(Q / 2))
T = cp_optimizer.init_matrix(q_list, res, C, precision)
probas = None#int(probas * 10000)

In [9]:
#Define program
model = cp_model.CpModel()
#Init Variable
V = {(q, r): model.NewIntVar(0, C, f'V_{q},{r}') for q in range(Q) for r in range(R)}


I = {(q, r): model.NewIntVar(0, R - 1, f'I_{q},{r}') for q in range(Q) for r in range(R)}
X = {q: model.NewIntVar(1, C, f'X_{q}') for q in range(Q)}
k = {r: model.NewIntVar(0, T.max(), f'k_{r}') for r in range(R)}
t_ind = {(q, r): model.NewIntVar(0, T.max(), f't_ind_{q},{r}') for q in range(Q) for r in range(R)}

A = np.zeros((Q, R)).astype(int).astype(object)
for q in range(Q):
    A[q, -1] = X[q]
#V, I, X, k, t_ind, A

#for probability approach
V_bool = {(q, r): model.NewBoolVar(f'V_bool_{q},{r}') for q in range(Q) for r in range(R)}
index_run = {q: model.NewIntVar(0, R - 1, f'index_run_{q}') for q in range(Q)}
#TODO :check if can reduce range
runtime_queries = {q: model.NewIntVar(T.min() * Q, T.max() * Q, f'run_q_{q}') for q in range(Q)}

In [65]:
def get_path_sets(q_list):
    results = []

    results.append(set([q_list[0]]))
    height = int(math.log2(len(q_list)))

    for i in range(height):
        for j in range(2**i):
            idx_parent = int(j + 2**i - 1)
            parent_set = results[idx_parent]
            
            idx_left = 2**(i + 1) + j*2 - 1
            left_set = parent_set.copy()
            left_set.add(q_list[idx_left])
            results.append(left_set)
            
            right_set = parent_set.copy()
            right_set.add(q_list[idx_left + 1])
            results.append(right_set)

    return results[2**(height)-1:]

In [4]:
#Define program
# 1
for q in range(Q):
    for r in range(R):
        model.AddElement(I[q, r], list(A[q, :]), V[q, r])
    model.AddAllDifferent([I[q, r] for r in range(R)])
# 2
for r in range(R):
    if C_ is None:
        model.Add(sum(V[q, r] for q in range(Q)) <= C)
    else:
        model.Add(sum(V[q, r] for q in range(Q)) == C_)
# 3
for q in range(Q):
    for r in range(R):
        model.AddElement(V[q, r], list(T[q, :]), t_ind[q, r])
for r in range(R):
    model.AddMaxEquality(k[r], [t_ind[q, r] for q in range(Q)])

if probas is None:
    obj = sum(k[r] for r in range(R))
else :
    #Get index of which run each query is
    for q in range(Q):
        for r in range(R):
            model.Add(V[q,r] > 0).OnlyEnforceIf(V_bool[q,r])
            model.Add(V[q,r] == 0).OnlyEnforceIf(V_bool[q,r].Not())

    for q in range(Q):
        model.Add(index_run[q]==0).OnlyEnforceIf([V_bool[q,r].Not() for r in range(R)]) 
        for r in range(R):
            model.Add(index_run[q]== r).OnlyEnforceIf(V_bool[q,r]) 
            
    #Get runtime of each query
    for q in range(Q):
        model.Add(runtime_queries[q] == sum(k[r] for r in range(index_run[q] + 1)))
    #TODO : implement
    # method to define paths
    obj = None

model.Minimize(obj)

In [5]:
start_time = time.time()
solver = cp_model.CpSolver()
status = solver.Solve(model)
process_time = time.time() - start_time

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # (V, _, _, k, _, _) = variables
    runtime, res_schedule = cp_optimizer.model_to_solution(solver, R, V, k, q_list, precision)
else:
    runtime, res_schedule = -1, []

process_time, runtime, res_schedule

(1.0365498065948486,
 39.9,
 [[('q1', 7), ('q4', 9)],
  [('q2', 8), ('q2', 8)],
  [('q3', 4), ('q5', 4), ('q6', 8)]])

## Sample test

In [6]:
[cp_model.UNKNOWN, cp_model.MODEL_INVALID,cp_model.FEASIBLE, cp_model.INFEASIBLE , cp_model.OPTIMAL]

[0, 1, 2, 3, 4]

In [7]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()
R = 7
Q = 6
V = {(q, r): model.NewIntVar(0, C, f'V_{q},{r}') for q in range(Q) for r in range(R)}
V_bool = {(q, r): model.NewBoolVar(f'V_bool_{q},{r}') for q in range(Q) for r in range(R)}
index_run = {q: model.NewIntVar(0, R - 1, f'index_run_{q}') for q in range(Q)}


for q in range(Q):
    for r in range(R):
        model.Add(V[q,r] > 0).OnlyEnforceIf(V_bool[q,r])
        model.Add(V[q,r] == 0).OnlyEnforceIf(V_bool[q,r].Not())

for q in range(Q):
    model.Add(index_run[q]==0).OnlyEnforceIf([V_bool[q,r].Not() for r in range(R)]) 
    for r in range(R):
        model.Add(index_run[q]== r).OnlyEnforceIf(V_bool[q,r]) 